In [2]:
import pandas as pd
import numpy as np
import tpqoa
from datetime import datetime, timedelta, timezone # Need to import timezone seperatly
import time

## The original ConTrader (functional)

In [5]:
class ConTrader(tpqoa.tpqoa):
    def __init__(self, conf_file, instrument, bar_length, window, units):
        super().__init__(conf_file)
        self.instrument = instrument
        self.bar_length = pd.to_timedelta(bar_length)
        self.tick_data = pd.DataFrame()
        self.raw_data = None
        self.data = None 
        self.last_bar = None
        self.units = units
        self.position = 0
        self.profits = [] 
        
        #*****************add strategy-specific attributes here******************
        self.window = window
        #************************************************************************
    
    def get_most_recent(self, days = 5):
        while True:
            time.sleep(2)
            now = datetime.now(timezone.utc).replace(tzinfo=None) # new (Python 3.12)
            now = now - timedelta(microseconds = now.microsecond)
            past = now - timedelta(days = days)
            df = self.get_history(instrument = self.instrument, start = past, end = now,
                                   granularity = "S5", price = "M", localize = False).c.dropna().to_frame()
            df.rename(columns = {"c":self.instrument}, inplace = True)
            df = df.resample(self.bar_length, label = "right").last().dropna().iloc[:-1]
            self.raw_data = df.copy()
            self.last_bar = self.raw_data.index[-1]
            if pd.to_datetime(datetime.now(timezone.utc)) - self.last_bar < self.bar_length: # new (Python 3.12)
                break
            
    def start_trading(self, days): # NEW
        
        self.get_most_recent(days)
        self.stream_data(self.instrument)
            
    def on_success(self, time, bid, ask):
        print(self.ticks, end = "\r", flush = True)
        
        recent_tick = pd.to_datetime(time)
        
        # define stop
        if self.ticks >= 100:
            self.terminate_session(cause = "Scheduled Session End.")
            return
        
        # collect and store tick data
        df = pd.DataFrame({self.instrument:(ask + bid)/2}, 
                          index = [recent_tick])
        self.tick_data = pd.concat([self.tick_data, df]) 
        
        # if a time longer than the bar_lenght has elapsed between last full bar and the most recent tick
        if recent_tick - self.last_bar >= self.bar_length:
            self.resample_and_join()
            self.define_strategy()
            self.execute_trades()
            
    def resample_and_join(self):
        self.raw_data = pd.concat([self.raw_data, self.tick_data.resample(self.bar_length, 
                                                                          label="right").last().ffill().iloc[:-1]])
        self.tick_data = self.tick_data.iloc[-1:]
        self.last_bar = self.raw_data.index[-1]
        
    def define_strategy(self): # "strategy-specific"
        df = self.raw_data.copy()
        
        #******************** define your strategy here ************************
        df["returns"] = np.log(df[self.instrument] / df[self.instrument].shift())
        df["position"] = -np.sign(df.returns.rolling(self.window).mean())
        #***********************************************************************
        
        self.data = df.copy()
        
    def execute_trades(self):
        if self.data["position"].iloc[-1] == 1:
            if self.position == 0:
                order = self.create_order(self.instrument, self.units, suppress = True, ret = True)
                self.report_trade(order, "GOING LONG")  
            elif self.position == -1:
                order = self.create_order(self.instrument, self.units * 2, suppress = True, ret = True) 
                self.report_trade(order, "GOING LONG")  
            self.position = 1
        elif self.data["position"].iloc[-1] == -1: 
            if self.position == 0:
                order = self.create_order(self.instrument, -self.units, suppress = True, ret = True)
                self.report_trade(order, "GOING SHORT")  
            elif self.position == 1:
                order = self.create_order(self.instrument, -self.units * 2, suppress = True, ret = True)
                self.report_trade(order, "GOING SHORT")  
            self.position = -1
        elif self.data["position"].iloc[-1] == 0: 
            if self.position == -1:
                order = self.create_order(self.instrument, self.units, suppress = True, ret = True) 
                self.report_trade(order, "GOING NEUTRAL")  
            elif self.position == 1:
                order = self.create_order(self.instrument, -self.units, suppress = True, ret = True)
                self.report_trade(order, "GOING NEUTRAL")  
            self.position = 0
    
    def report_trade(self, order, going):  
        time = order["time"]
        units = order["units"]
        price = order["price"]
        pl = float(order["pl"])
        self.profits.append(pl)
        cumpl = sum(self.profits)
        print("\n" + 100* "-")
        print("{} | {}".format(time, going))
        print("{} | units = {} | price = {} | P&L = {} | Cum P&L = {}".format(time, units, price, pl, cumpl))
        print(100 * "-" + "\n")  
        
    def terminate_session(self, cause): # NEW
        self.stop_stream = True
        if self.position != 0:
            close_order = self.create_order(self.instrument, units = -self.position * self.units,
                                            suppress = True, ret = True) 
            self.report_trade(close_order, "GOING NEUTRAL")
            self.position = 0
        print(cause)

In [7]:
trader = ConTrader("oanda.cfg", "EUR_USD", "1min", window = 1, units = 100)

In [9]:
trader.start_trading(days = 5)

C:\Users\audri\anaconda3\Lib\site-packages\tpqoa\tpqoa.py:223: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  dr = pd.date_range(start, end, freq=freq)


1
----------------------------------------------------------------------------------------------------
2025-04-23T05:13:11.000941815Z | GOING LONG
2025-04-23T05:13:11.000941815Z | units = 100.0 | price = 1.13956 | P&L = 0.0 | Cum P&L = 0.0
----------------------------------------------------------------------------------------------------

43
----------------------------------------------------------------------------------------------------
2025-04-23T05:14:00.376510736Z | GOING SHORT
2025-04-23T05:14:00.376510736Z | units = -200.0 | price = 1.1393 | P&L = 20.342 | Cum P&L = 20.342
----------------------------------------------------------------------------------------------------

100
----------------------------------------------------------------------------------------------------
2025-04-23T05:14:59.774356282Z | GOING NEUTRAL
2025-04-23T05:14:59.774356282Z | units = 100.0 | price = 1.13946 | P&L = 0.0 | Cum P&L = 20.342
------------------------------------------------------------

## Expanding a error handling block start_trading

In [14]:
class ConTrader(tpqoa.tpqoa):
    def __init__(self, conf_file, instrument, bar_length, window, units):
        super().__init__(conf_file)
        self.instrument = instrument
        self.bar_length = pd.to_timedelta(bar_length)
        self.tick_data = pd.DataFrame()
        self.raw_data = None
        self.data = None 
        self.last_bar = None
        self.units = units
        self.position = 0
        self.profits = [] 
        
        #*****************add strategy-specific attributes here******************
        self.window = window
        #************************************************************************
    
    def get_most_recent(self, days = 5):
        while True:
            time.sleep(2)
            now = datetime.now(timezone.utc).replace(tzinfo=None) # new (Python 3.12)
            now = now - timedelta(microseconds = now.microsecond)
            past = now - timedelta(days = days)
            df = self.get_history(instrument = self.instrument, start = past, end = now,
                                   granularity = "S5", price = "M", localize = False).c.dropna().to_frame()
            df.rename(columns = {"c":self.instrument}, inplace = True)
            df = df.resample(self.bar_length, label = "right").last().dropna().iloc[:-1]
            self.raw_data = df.copy()
            self.last_bar = self.raw_data.index[-1]
            if pd.to_datetime(datetime.now(timezone.utc)) - self.last_bar < self.bar_length: # new (Python 3.12)
                break
            
    def start_trading(self, days, max_attempts = 5, wait = 20, wait_increase = 0): # NEW, erroe handling
        attempt = 0
        success = False 
        while True:
            try:
                self.get_most_recent(days)
                self.stream_data(self.instrument)
            except Exceptions as e:
                print(e, end = " | ")
            else:
                success = True
                break
            finally:
                attempt += 1
                print("Attempt: {}".format(attempt), end = 'n/')
                if success == False:
                    if attempt >= max_attempts:
                        print("Max_attempts reached!")
                        try: # for termination 
                            time.sleep(wait)
                            self.terminate_session(cause = "Unexpected Session Stop (too many errors).")
                        except Exception as e:
                            print(e, end = " | ")
                            print("Could not terminate the session.")
                        finally:
                            break
                    else:
                        time.sleep(wait)
                        wait += wait_increase
                        self.tick_data = pd.DataFrame()
                     
    def on_success(self, time, bid, ask):
        print(self.ticks, end = "\r", flush = True)
        
        recent_tick = pd.to_datetime(time)
        
        # define stop
        if self.ticks >= 100:
            self.terminate_session(cause = "Scheduled Session End.")
            return
        
        # collect and store tick data
        df = pd.DataFrame({self.instrument:(ask + bid)/2}, 
                          index = [recent_tick])
        self.tick_data = pd.concat([self.tick_data, df]) 
        
        # if a time longer than the bar_lenght has elapsed between last full bar and the most recent tick
        if recent_tick - self.last_bar >= self.bar_length:
            self.resample_and_join()
            self.define_strategy()
            self.execute_trades()
            
    def resample_and_join(self):
        self.raw_data = pd.concat([self.raw_data, self.tick_data.resample(self.bar_length, 
                                                                          label="right").last().ffill().iloc[:-1]])
        self.tick_data = self.tick_data.iloc[-1:]
        self.last_bar = self.raw_data.index[-1]
        
    def define_strategy(self): # "strategy-specific"
        df = self.raw_data.copy()
        
        #******************** define your strategy here ************************
        df["returns"] = np.log(df[self.instrument] / df[self.instrument].shift())
        df["position"] = -np.sign(df.returns.rolling(self.window).mean())
        #***********************************************************************
        
        self.data = df.copy()
        
    def execute_trades(self):
        if self.data["position"].iloc[-1] == 1:
            if self.position == 0:
                order = self.create_order(self.instrument, self.units, suppress = True, ret = True)
                self.report_trade(order, "GOING LONG")  
            elif self.position == -1:
                order = self.create_order(self.instrument, self.units * 2, suppress = True, ret = True) 
                self.report_trade(order, "GOING LONG")  
            self.position = 1
        elif self.data["position"].iloc[-1] == -1: 
            if self.position == 0:
                order = self.create_order(self.instrument, -self.units, suppress = True, ret = True)
                self.report_trade(order, "GOING SHORT")  
            elif self.position == 1:
                order = self.create_order(self.instrument, -self.units * 2, suppress = True, ret = True)
                self.report_trade(order, "GOING SHORT")  
            self.position = -1
        elif self.data["position"].iloc[-1] == 0: 
            if self.position == -1:
                order = self.create_order(self.instrument, self.units, suppress = True, ret = True) 
                self.report_trade(order, "GOING NEUTRAL")  
            elif self.position == 1:
                order = self.create_order(self.instrument, -self.units, suppress = True, ret = True)
                self.report_trade(order, "GOING NEUTRAL")  
            self.position = 0
    
    def report_trade(self, order, going):  
        time = order["time"]
        units = order["units"]
        price = order["price"]
        pl = float(order["pl"])
        self.profits.append(pl)
        cumpl = sum(self.profits)
        print("\n" + 100* "-")
        print("{} | {}".format(time, going))
        print("{} | units = {} | price = {} | P&L = {} | Cum P&L = {}".format(time, units, price, pl, cumpl))
        print(100 * "-" + "\n")  
        
    def terminate_session(self, cause): # NEW
        self.stop_stream = True
        if self.position != 0:
            close_order = self.create_order(self.instrument, units = -self.position * self.units,
                                            suppress = True, ret = True) 
            self.report_trade(close_order, "GOING NEUTRAL")
            self.position = 0
        print(cause)

In [16]:
trader = ConTrader("Oanda.cfg", "EUR_USD", "1min", window = 1, units = 100)

In [18]:
trader.start_trading(days = 5, max_attempts = 3, wait = 20, wait_increase = 0)

C:\Users\audri\anaconda3\Lib\site-packages\tpqoa\tpqoa.py:223: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  dr = pd.date_range(start, end, freq=freq)


1
----------------------------------------------------------------------------------------------------
2025-04-23T05:15:41.525597020Z | GOING LONG
2025-04-23T05:15:41.525597020Z | units = 100.0 | price = 1.13945 | P&L = 0.0 | Cum P&L = 0.0
----------------------------------------------------------------------------------------------------

100
----------------------------------------------------------------------------------------------------
2025-04-23T05:17:47.265544646Z | GOING NEUTRAL
2025-04-23T05:17:47.265544646Z | units = -100.0 | price = 1.13928 | P&L = 10.169 | Cum P&L = 10.169
----------------------------------------------------------------------------------------------------

Scheduled Session End.
Attempt: 1n/

In [17]:
trader.data.tail(15)

,EUR_USD,returns,position
2025-04-15 00:23:00+00:00,1.13314,0.000088,-1.0
2025-04-15 00:24:00+00:00,1.13304,-0.000088,1.0
2025-04-15 00:25:00+00:00,1.13308,0.000035,-1.0
2025-04-15 00:26:00+00:00,1.13306,-0.000018,1.0
2025-04-15 00:27:00+00:00,1.13270,-0.000318,1.0
2025-04-15 00:28:00+00:00,1.13261,-0.000079,1.0
2025-04-15 00:29:00+00:00,1.13270,0.000079,-1.0
2025-04-15 00:30:00+00:00,1.13264,-0.000053,1.0
2025-04-15 00:31:00+00:00,1.13258,-0.000053,1.0
2025-04-15 00:32:00+00:00,1.13253,-0.000044,1.0
